In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from pprint import pprint as pp
import csv
from pathlib import Path
import itertools
from itertools import product, combinations, permutations
import random
from datetime import datetime, timedelta, date
from collections import OrderedDict, defaultdict, Counter, namedtuple
from pandas.io.json import json_normalize
import seaborn as sns
import matplotlib.dates as mdates
import re
import requests
from bs4 import BeautifulSoup
import statsmodels.api as sm
import calendar
import scipy
import json
from PIL import Image
from bs4 import BeautifulSoup as bs
from ast import literal_eval  # use to eval a string as a list df['column'] = df['column'].apply(literal_eval)
from stockstats import StockDataFrame as sdf  # dataframe wrapper for stack calculations
import math
import time
import sys
import string
import json
import sklearn
import urllib
import urllib3

In [ ]:
pd.set_option('display.max_columns', 700)
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 10)
pd.set_option('display.expand_frame_repr', True)

## Synthetic Data:

### Use recursion to flatten the nested `dicts`

 - [Thinking Recursively in Python](https://realpython.com/python-thinking-recursively/)
 - [Flattening JSON objects in Python](https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10)
 - [flatten](https://github.com/amirziai/flatten)
 - The `flatten_json` function, will be used to flatten `data`

In [ ]:
def flatten_json(nested_json: dict, exclude: list=[''], sep='_') -> dict:
    """
    Flatten a list of nested dicts.
    """
    out = dict()
    def flatten(x: (list, dict, str), name: str='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude:
                    flatten(x[a], f'{name}{a}{sep}')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, f'{name}{i}{sep}')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

# df = pd.DataFrame([flatten_json(x) for x in data])
# df = pd.DataFrame([flatten_json(x) for x in data[key]])

In [ ]:
np.random.seed(365)

In [ ]:
date_0 = pd.date_range(pd.datetime.today(), periods=10).to_pydatetime().tolist()
date_0[:2]

In [ ]:
date_1 = pd.date_range(end=pd.datetime.today(), periods=10).to_pydatetime().tolist()
date_1[:2]

In [ ]:
date = pd.bdate_range(end=pd.datetime.today(), periods=10).to_pydatetime().tolist()
date[:2]

In [ ]:
np.random.seed(365)
data = {'a': [np.random.randint(10) for _ in range(100)],
        'b': [np.random.randint(10) for _ in range(100)],
        'c': [np.random.randint(10) for _ in range(100)],
        'd': [np.random.randint(10) for _ in range(100)],
        'e': [np.random.randint(10) for _ in range(100)],
        'date': pd.bdate_range(pd.datetime.today(), periods=100).tolist()}

In [ ]:
random.seed(365)
data = {'groups': [random.choice(['1-5', '6-25', '26-100', '100-500', '500-1000', '>1000']) for _ in range(1000)],
        'treatment': [random.choice(['Yes', 'No']) for _ in range(1000)]}

In [ ]:
# stock data
periods = '3600'
resp = requests.get('https://api.cryptowat.ch/markets/poloniex/ethusdt/ohlc', params={'periods': periods})
data = resp.json()
df = pd.DataFrame(data['result'][periods], columns=['date', 'open', 'high', 'low', 'close', 'volume', 'amount'])
df['date'] = pd.to_datetime(df['date'], unit='s')

stock = sdf.retype(df)
stock['macds']
# stock

In [ ]:
# p = Path('test.json')  # if in current dir
# p = Path.cwd() / 'test.json'
p = Path.cwd() / 'data/nvdcve-1.1-2019.json/nvdcve-1.1-2019.json'

In [ ]:
with p.open('r') as f:
    data = json.loads(f.read())

In [ ]:
# Use with a list of dicts
with p.open("r") as f:
    data = literal_eval(f.read())

In [ ]:
df = json_normalize(data, record_path='trends')

In [ ]:
df = pd.read_clipboard(sep='\\s+')

In [ ]:
df.head(2).to_clipboard(sep='\\s+', index=False)

In [ ]:
df = pd.read_csv('test.csv', parse_dates=True)  # converters={'file_path_lists': eval}

In [ ]:
df = pd.read_excel('test.xlsx')

In [ ]:
df = pd.DataFrame(data)

In [ ]:
# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']  # list a variables

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

## End Synthetic Data:

In [ ]:
f = Path.cwd() / 'household_energy_usage_data.csv'

In [ ]:
df = pd.read_csv(f)

In [ ]:
df

In [ ]:
regions = {1: 'Northeast', 2: 'Midwest', 3: 'South', 4: 'West'}

In [ ]:
df['REGION_CAT'] = df['REGIONC'].map(regions)

In [ ]:
reg_type = df.groupby(['REGION_CAT', 'UATYP10'])['NWEIGHT'].sum().unstack().reset_index().melt(id_vars='REGION_CAT')
reg_type

In [ ]:
sns.barplot(x='REGION_CAT', y='value', data=reg_type, hue='UATYP10')
plt.show()

In [ ]:
df.NWEIGHT

In [ ]:
fuel_heat = df[df.FUELHEAT == 1]
fuel_heat

In [ ]:
fuel_heat['NWEIGHT'].sum().round(2)

1. Correlation between square footage and energy use, using weights
1. Age of appliances, ENERGYSTAR and how it affects energy usage
1. Does "who pays" matter in energy consumption?
1. Effects of basements and attics
1. Linear regression
1. What factor(s) best predict energy consumption?
1. Lasso method to determine most important factors; dimensionality reduction
1. Predict other characteristics of household (e.g. heating type)
1. Which are the most cost effective energy savings method?  When compared with normalized mean consumption.
1. Have energy assistance programs (e.g. lower temperature) caused less hardship?
1. Predict "sad" columns from characteristics such as household temperature.
1. Is there a role for education programs as opposed to supplementing the energy bill?
1. Data cleaning
1. Predict number of "things" based on energy use?
1. Can household characteristics predict who is having trouble with bills?
1. Relationship between income and energy?
1. Are imputed values more common for some columns?

In [ ]:
df[['CDD30YR', 'CDD65', 'CDD80']]